In [109]:
import subprocess
import yaml
from time import sleep
import os
import numpy as np

In [ ]:
def ticcmd(*args):
    return subprocess.check_output(['ticcmd'] + list(args))

def get_position():
    status = yaml.load(ticcmd('-s', '--full'))
    position = status['Current position']
    return position

def set_position(target_position):
    position = get_position()
    ticcmd('--resume', '--position', str(target_position))
    while position != target_position:
        position = get_position()
        print(position, end="\r")
        sleep(0.1)
    #sleep(0.5)
    #print(position)
    
def set_cavity_frequency(current_frequency, target_frequency, calibration_constant):
    frequency_change = target_frequency - current_frequency
    steps_required = int(np.rint(frequency_change / calibration_constant))
    target_position = get_position() + steps_required
    set_position(target_position)
    
def reset_position(initial_position):
    set_position(initial_position - 500)
    set_position(initial_position)

In [22]:
initial_frequency = 2
initial_position = get_position()
frequencies = [initial_frequency, 2.6, 2.7, 2.8, 2.9]
counts = [0] * len(frequencies)
calibration_constant = 0.00015094
number_of_runs = 10

In [16]:
for i in range(number_of_runs):
    for i in range(len(frequencies)):
        if i == 0:
            target_frequency = frequencies[i]
            current_frequency = frequencies[i]
            smiq.set_cw(frequency=target_frequency)
            smiq.cw_on()
            counterlogic.startCount()
            counterlogic.countdata
            counts[i] = counts[i] + sum(counterlogic.countdata)
            smiq.off()

        if i == len(frequencies):
            target_frequency = frequencies[i]
            current_frequency = frequencies[i - 1]
            set_cavity_frequency(current_frequency, target_frequency, calibration_constant)
            smiq.set_cw(frequency=target_frequency)
            smiq.cw_on()
            counterlogic.startCount()
            counterlogic.countdata
            counts[i] = counts[i] + sum(counterlogic.countdata)
            smiq.off()
            reset_position(initial_position)

        else:
            target_frequency = frequencies[i]
            current_frequency = frequencies[i - 1]
            set_cavity_frequency(current_frequency, target_frequency, calibration_constant)
            smiq.set_cw(frequency=target_frequency)
            smiq.cw_on()
            counterlogic.startCount()
            counterlogic.countdata
            counts[i] = counts[i] + sum(counterlogic.countdata)
            smiq.off()

2.6 2.6
2.7 2.6
2.8 2.7
2.9 2.8


In [ ]:
#Sets current position to 0
zero_position = 0
ticcmd('--exit-safe-start', '--halt-and-set-position', str(zero_position))

In [ ]:
# Brute force way to control anything, pull gui levers.
counter._mw.start_counter_Action.trigger()

In [ ]:
#Get current count parameters
counterlogic.get_count_length()
counterlogic.get_count_frequency()

In [ ]:
#Set count parameters
counterlogic.set_count_length()
counterlogic.set_count_frequency()